# Calculate NDVI

In [9]:
import ee
import folium

In [2]:
# ee.Authenticate()
ee.Initialize()

### Load Country Feature

In [12]:
countries = ee.FeatureCollection('FAO/GAUL/2015/level0')
Argentina = countries.filter(ee.Filter.eq('ADM0_NAME', 'Argentina'))

### Load Landsat Data

In [13]:
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
.filterDate('2020-01-01','2020-12-31')\
.filterBounds(Argentina)\
.median()

### Compute NDVI

In [16]:
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4'])

NDVI_Palette = '00FF00', 'FFFF00', '000000'

In [17]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[-48, -70], zoom_start = 6)

# Display
my_map.add_ee_layer(ndvi.clip(Argentina), {'min': -0.3, 'max': 0.5, 'palette': NDVI_Palette}, 'NDVI')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)